## Red Hat Product Documentation Verification

Using this notebook and the various dependencies, you can verify that the documentation was ingested properly inside the corresponding Milvus collection.

In [ ]:
import os
import json
from langchain_openai import OpenAIEmbeddings
from langchain_milvus import Milvus

In [ ]:
class product_info:
    def __init__(self, product, product_full_name, version, language):
        self.product = product
        self.product_full_name = product_full_name
        self.version = version
        self.language = language

    def __repr__(self):
        return f"product_info(product={self.product}, product_full_name={self.product_full_name}, version={self.version}, language={self.language})"

In [ ]:
# Load collections from JSON file
with open('collections.json', 'r') as file:
    data = json.load(file)

# Iterate over the products and versions
products = []
for item in data:
    product = item['product']
    product_full_name = item['product_full_name']
    language = item['language']

    for version in item['version']:
        # Create an instance of the product_info class and append to the list
        products.append(product_info(product, product_full_name, version, language))

In [ ]:
milvus = {}
milvus["MILVUS_HOST"] = os.getenv('MILVUS_HOST')
milvus["MILVUS_PORT"] = os.getenv('MILVUS_PORT')
milvus["MILVUS_USERNAME"] = os.getenv('MILVUS_USERNAME')
milvus["MILVUS_PASSWORD"] = os.getenv('MILVUS_PASSWORD')
embeddings_endpoint = {}
embeddings_endpoint["EMBEDDINGS_API_URL"] = os.getenv('EMBEDDINGS_API_URL')
embeddings_endpoint["EMBEDDINGS_API_KEY"] = os.getenv('EMBEDDINGS_API_KEY')

In [ ]:
embeddings = OpenAIEmbeddings(
        openai_api_base=embeddings_endpoint["EMBEDDINGS_API_URL"],
        openai_api_key=embeddings_endpoint["EMBEDDINGS_API_KEY"],
        model="nomic-embed-text-v1.5",
        chunk_size="16",
        show_progress_bar=False
    )

In [ ]:
query = "How do I install this product?"

for product_info in products:
    MILVUS_COLLECTION = f"{product_info.product}_{product_info.language}_{product_info.version}".replace("-","_").replace(".","_")
    vector_store = Milvus(
        embedding_function=embeddings,
        connection_args={"host": milvus["MILVUS_HOST"], "port": milvus["MILVUS_PORT"], "user": milvus["MILVUS_USERNAME"], "password": milvus["MILVUS_PASSWORD"]},
        collection_name=MILVUS_COLLECTION,
        enable_dynamic_field=True,
        text_field="page_content",
        auto_id=True,
        drop_old=False
        )
    docs_with_score = vector_store.similarity_search_with_score(query)

    for doc, score in docs_with_score:
        print("-" * 80)
        print("Score: ", score)
        print(doc.page_content)
        print("-" * 80)